# This Notebook is made to conduct a T-Test to show if different Scheduling have a statistical significance on the nDCG and or on the rND

In [1]:
import pandas as pd
import scipy.stats as stats
import statsmodels.stats.power as smp
from resultsHelper import cohen_d, perform_shapiro_test

In [9]:
datasets = ['Adult', 'Compas', 'Law-Race', 'Law-Gender']
metrics = ['nDCG', 'rND', 'GPA']

In [10]:
total_normal = 0
total_not_normal = 0
for dataset in datasets:
    df = pd.read_csv(f'../Results/DirectRankerAdv/{dataset}/T-Test/Run02/results.csv')
    count_normal = 0
    count_not_normal = 0
    for metric in metrics:
        for schedule in df['Schedule'].unique():
            stat, p_value = stats.shapiro(df[df['Schedule'] == schedule][metric])

            if p_value < 0.05:
                count_not_normal += 1
            else:
                count_normal += 1
    print(f'{dataset} - normal: {count_normal}  not normal: {count_not_normal}')
    total_normal += count_normal
    total_not_normal += count_not_normal

print(f'Total normal: {total_normal}  total not normal: {total_not_normal}')


Adult - normal: 20  not normal: 7
Compas - normal: 22  not normal: 5
Law-Race - normal: 24  not normal: 3
Law-Gender - normal: 17  not normal: 10
Total normal: 83  total not normal: 25


In [21]:
df = pd.read_csv('../Results/DebiasClassifier/Adult/T-Test/results.csv')
# Lists containing the performance metric for each schedule
print(df['Schedule'].unique())

['[1; 2]' '[2; 1]' '[1; 3]' '[3; 1]' '[1; 4]' '[4; 1]' '[1; 5]' '[5; 1]'
 '[1; 1]']


In [24]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Load your data
# data = pd.read_csv('your_data.csv')

# Fit the model
model = ols('GPA ~ C(Schedule)', data=df).fit()

# Perform ANOVA
anova_results = sm.stats.anova_lm(model, typ=2)

# Print the results
print(anova_results)

               sum_sq     df          F        PR(>F)
C(Schedule)  0.173363    8.0  10.484879  9.630910e-13
Residual     0.539440  261.0        NaN           NaN


In [28]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import statsmodels.api as sm

tukey = pairwise_tukeyhsd(endog=df['nDCG'],     
                          groups=df['Schedule'],    
                          alpha=0.05)          


tukey_df = pd.DataFrame(data=tukey._results_table.data[1:], columns=tukey._results_table.data[0])
print(np.unique(tukey_df['reject'], return_counts=True))

(array([False,  True]), array([35,  1]))


In [26]:
import numpy as np
def get_req_samples(row, metric):
    schedule_a = list(df[df['Schedule']==row['group1']][metric])
    schedule_b = list(df[df['Schedule']==row['group2']][metric])
    d = cohen_d(schedule_a, schedule_b)
    effect_size = d
    alpha = 0.05       
    power = 0.8        
    ratio = 1          
    sample_size = smp.TTestIndPower().solve_power(effect_size=effect_size,
                                                power=power,
                                                alpha=alpha,
                                                ratio=ratio,
                                                alternative='two-sided')
    if type(sample_size) == np.ndarray:
        return round(sample_size[0],2)
    return round(sample_size, 2)

In [267]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import statsmodels.api as sm
full_tukey = pd.DataFrame()
for metric in ['nDCG', 'rND', 'GPA']:
    tukey = pairwise_tukeyhsd(endog=df[metric],     
                            groups=df['Schedule'],    
                            alpha=0.05)          
    tukey_df = pd.DataFrame(data=tukey._results_table.data[1:], columns=tukey._results_table.data[0])
    tukey_df['needed samples'] = tukey_df.apply(get_req_samples, args=(metric,), axis=1)
    full_tukey = pd.concat([full_tukey, tukey_df], axis=0)
full_tukey.to_csv('tukey_hsd_result_adult.csv', index=False)

/opt/anaconda3/envs/pytorchbook/lib/python3.10/site-packages/statsmodels/stats/power.py:525: ConvergenceWarning: 
Failed to converge on a solution.

  warnings.warn(convergence_doc, ConvergenceWarning)
/opt/anaconda3/envs/pytorchbook/lib/python3.10/site-packages/statsmodels/stats/power.py:525: ConvergenceWarning: 
Failed to converge on a solution.

  warnings.warn(convergence_doc, ConvergenceWarning)
/opt/anaconda3/envs/pytorchbook/lib/python3.10/site-packages/statsmodels/stats/power.py:525: ConvergenceWarning: 
Failed to converge on a solution.

  warnings.warn(convergence_doc, ConvergenceWarning)
/opt/anaconda3/envs/pytorchbook/lib/python3.10/site-packages/statsmodels/stats/power.py:525: ConvergenceWarning: 
Failed to converge on a solution.

  warnings.warn(convergence_doc, ConvergenceWarning)
/opt/anaconda3/envs/pytorchbook/lib/python3.10/site-packages/statsmodels/stats/power.py:525: ConvergenceWarning: 
Failed to converge on a solution.

  warnings.warn(convergence_doc, Convergenc

In [228]:
a = '[1; 3]'
b = '[3; 1]'
schedule_a = list(df[df['Schedule']==a]['rND'])
schedule_b = list(df[df['Schedule']==b]['rND'])

In [175]:
df[df['Schedule']=='[4; 1]'].describe()

,nDCG,AUC,rND,GPA
count,30.000000,30.000000,30.000000,30.000000
mean,0.894415,0.896448,0.320257,0.109946
std,0.019164,0.003807,0.016197,0.015905
min,0.858409,0.886894,0.293366,0.073573
25%,0.881513,0.894201,0.308873,0.102632
50%,0.897801,0.897537,0.319651,0.108673
75%,0.908295,0.899232,0.331910,0.120682
max,0.931348,0.904315,0.357845,0.141350


### To check if the data is normal the Shapiro Wilk test will be performed

In [5]:
# Perform the test for nDCG
perform_shapiro_test(df, 'AUC')

perform_shapiro_test(df, 'Accuracy')

# Perform the test for rND
perform_shapiro_test(df, 'rND')

# Perform the test for GPA
perform_shapiro_test(df, 'GPA')

Shapiro-Wilk Test for AUC in Schedule [1; 1]: Statistic=0.9349406361579895, p-value=0.06651399284601212
	Null hypothesis not rejected - data is normal
Shapiro-Wilk Test for AUC in Schedule [1; 2]: Statistic=0.9599834680557251, p-value=0.3094736635684967
	Null hypothesis not rejected - data is normal
Shapiro-Wilk Test for AUC in Schedule [2; 1]: Statistic=0.9556882977485657, p-value=0.23942343890666962
	Null hypothesis not rejected - data is normal
Shapiro-Wilk Test for AUC in Schedule [1; 3]: Statistic=0.8904260993003845, p-value=0.004937667399644852
	Null hypothesis rejected - data is not normal
Shapiro-Wilk Test for AUC in Schedule [3; 1]: Statistic=0.9857069849967957, p-value=0.958173930644989
	Null hypothesis not rejected - data is normal
Shapiro-Wilk Test for Accuracy in Schedule [1; 1]: Statistic=0.9711225032806396, p-value=0.570326566696167
	Null hypothesis not rejected - data is normal
Shapiro-Wilk Test for Accuracy in Schedule [1; 2]: Statistic=0.8644814491271973, p-value=0.00

### Now calculate Cohen's d to validate that the sample size is enough

In [265]:
a = '[1; 1]'
b = '[1; 3]'
schedule_a = list(df[df['Schedule']==a]['rND'])
schedule_b = list(df[df['Schedule']==b]['rND'])
d = cohen_d(schedule_a, schedule_b)
print("Cohen's d:", d)
effect_size = d
alpha = 0.05       
power = 0.8        
ratio = 1          

# Calculate the sample size
sample_size = smp.TTestIndPower().solve_power(effect_size=effect_size,
                                              power=power,
                                              alpha=alpha,
                                              ratio=ratio,
                                              alternative='two-sided')

print(f"Required sample size: {sample_size:.2f} per group")

Cohen's d: 6.738201893522704


/opt/anaconda3/envs/pytorchbook/lib/python3.10/site-packages/statsmodels/stats/power.py:525: ConvergenceWarning: 
Failed to converge on a solution.

  warnings.warn(convergence_doc, ConvergenceWarning)


TypeError: unsupported format string passed to numpy.ndarray.__format__

In [222]:
#schedule_a = list(df[df['Schedule']==a]['GPA'])
#schedule_b = list(df[df['Schedule']==b]['GPA'])
# Perform the t-test
t_stat, p_value = stats.ttest_ind(schedule_a, schedule_b)

# Check significance
alpha = 0.05
print(p_value)
if p_value < alpha:
    print("Reject the null hypothesis - suggest the means are different")
else:
    print("Do not reject the null hypothesis - no suggestion that the means are different")

0.00030458076014656883
Reject the null hypothesis - suggest the means are different


In [111]:
df = pd.read_csv('../Results/DirectRankerAdv/Adult/Schedule02/results.csv')
schedule_a = df['rND']
schedule_b = df['nDCG']
schedule_a.corr(schedule_b)

0.1539389090297381

In [132]:
import numpy as np
datasets = ['Adult', 'Compas', 'Law-Gender']#, 'Law-Race']
full_df = pd.DataFrame(columns=[
    'Dataset',
    'Schedule',
    'Cohen rND',
    'Cohen nDCG',
    'Cohen GPA',
    'p rND',
    'p nDCG',
    'p GPA',
    'mean GPA',
    'mean rND',
    'mean nDCG'
    ])
full_df.set_index('Dataset', inplace=True)
for dataset in datasets:
    df = pd.read_csv(f'../Results/DirectRankerAdv/{dataset}/T-Test/results.csv')
    means_a = {}
    means_b = {}
    ps = {}
    cohens = {}
    for metric in ['rND', 'nDCG', 'GPA']:
        schedule_a = list(df[df['Schedule']==a][metric])
        schedule_b = list(df[df['Schedule']==b][metric])
        cohens[metric] = cohen_d(schedule_a, schedule_b)
        print(dataset, metric, cohen_d(schedule_a, schedule_b))
        t_stat, p_value = stats.ttest_ind(schedule_a, schedule_b)
        ps[metric] = p_value
        means_a[metric] = np.mean(schedule_a)
        means_b[metric] = np.mean(schedule_b)
    new_row_a = pd.DataFrame({
            'Dataset': [dataset],
            'Schedule': [a],
            'Cohen rND': [cohens['rND']],
            'Cohen nDCG': [cohens['nDCG']],
            'Cohen GPA': [cohens['GPA']],
            'p rND': [ps['rND']],
            'p nDCG': [ps['nDCG']],
            'p GPA': [ps['GPA']],
            'mean GPA': [means_a['GPA']],
            'mean rND': [means_a['rND']],
            'mean nDCG': [means_a['nDCG']]
    }
    )
    new_row_a.set_index('Dataset', inplace=True)
    new_row_b = pd.DataFrame({
            'Dataset': [dataset],
            'Schedule': [b],
            'Cohen rND': [cohens['rND']],
            'Cohen nDCG': [cohens['nDCG']],
            'Cohen GPA': [cohens['GPA']],
            'p rND': [ps['rND']],
            'p nDCG': [ps['nDCG']],
            'p GPA': [ps['GPA']],
            'mean GPA': [means_b['GPA']],
            'mean rND': [means_b['rND']],
            'mean nDCG': [means_b['nDCG']]
    }
    )
    new_row_b.set_index('Dataset', inplace=True)
    full_df = pd.concat([full_df, new_row_a])
    full_df = pd.concat([full_df, new_row_b])
full_df.to_csv('./comparingDatasets.csv', sep=',')
        


Adult rND -1.9202137804038244
Adult nDCG -0.17682935945225622
Adult GPA -1.334112554679777
Compas rND 0.26386589595727755
Compas nDCG 0.7686730256062904
Compas GPA -0.09333426672490412
Law-Gender rND 0.16076896224569637
Law-Gender nDCG -0.47478062125078135
Law-Gender GPA 0.20609377504365367


In [135]:
test_df = pd.read_csv('./comparingDatasets.csv')
test_df.describe()

,Cohen rND,Cohen nDCG,Cohen GPA,p rND,p nDCG,p GPA,mean GPA,mean rND,mean nDCG
count,6.000000,6.000000,6.000000,6.000000e+00,6.000000,6.000000,6.000000,6.000000,6.000000
mean,-0.498526,0.039021,-0.407118,2.823335e-01,0.190489,0.382356,0.079502,0.215305,0.790362
std,1.102199,0.580681,0.730427,2.407147e-01,0.238652,0.323508,0.072348,0.123240,0.104500
min,-1.920214,-0.474781,-1.334113,5.402756e-10,0.004241,0.000003,0.002240,0.055451,0.654984
25%,-1.399968,-0.400293,-1.023918,7.776229e-02,0.020946,0.107005,0.017392,0.112006,0.707240
50%,0.160769,-0.176829,-0.093334,3.110492e-01,0.071063,0.428012,0.073068,0.284929,0.812608
75%,0.238092,0.532297,0.131237,4.797259e-01,0.389887,0.646292,0.142571,0.298402,0.873830
max,0.263866,0.768673,0.206094,5.359515e-01,0.496162,0.719053,0.164061,0.309265,0.895621
